In [1]:
%pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os


In [3]:
import torch

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")

# If CUDA is available, print the number of GPUs
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")

    # Print the name of the current GPU (device 0)
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")

    # Optional: Print properties of all available GPUs
    print("\nGPU Properties:")
    for i in range(torch.cuda.device_count()):
        print(f"  Device {i}: {torch.cuda.get_device_name(i)}")
        properties = torch.cuda.get_device_properties(i)
        print(f"    Memory: {properties.total_memory / (1024**3):.2f} GB")
        print(f"    Compute Capability: {properties.major}.{properties.minor}")
else:
    print("PyTorch is running on CPU.")

# You can also check the default device type
print(f"\nDefault device type: {torch.get_default_dtype()}")

# Try to move a tensor to the GPU
if torch.cuda.is_available():
    x = torch.tensor([1.0, 2.0, 3.0])
    print(f"\nCPU tensor: {x}")
    x_cuda = x.cuda() # or x.to('cuda')
    print(f"CUDA tensor: {x_cuda}")
    print(f"CUDA tensor device: {x_cuda.device}")

CUDA available: True
Number of GPUs: 1
Current GPU name: NVIDIA GeForce RTX 2060

GPU Properties:
  Device 0: NVIDIA GeForce RTX 2060
    Memory: 5.61 GB
    Compute Capability: 7.5

Default device type: torch.float32

CPU tensor: tensor([1., 2., 3.])
CUDA tensor: tensor([1., 2., 3.], device='cuda:0')
CUDA tensor device: cuda:0


In [4]:

class SD15ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5", use_cuda=True, num_inference_steps=25):
        """
        Initialize the Stable Diffusion 1.5 pipeline and inference settings.
        """
        self.device = "cuda" if use_cuda and torch.cuda.is_available() else "cpu"
        self.num_inference_steps = num_inference_steps
        self.intermediate_images = []

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            safety_checker=None
        ).to(self.device)

    def _capture_step(self, step, timestep, latents):
        """
        Internal callback to capture the image at each step.
        """
        # Decode latent to image at this step
        with torch.no_grad():
            image = self.pipe.vae.decode(latents / self.pipe.vae.config.scaling_factor).sample
            image = (image / 2 + 0.5).clamp(0, 1)
            image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
            image_pil = Image.fromarray((image * 255).astype("uint8"))
            self.intermediate_images.append(image_pil)

    def generate_image(self, prompt, negative_prompt=None, guidance_scale=7.5):
        """
        Generate image and collect intermediate steps.
        Returns a list of PIL images (one per step).
        """
        self.intermediate_images = []

        with torch.autocast(self.device) if self.device == "cuda" else torch.no_grad():
            _ = self.pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                guidance_scale=guidance_scale,
                num_inference_steps=self.num_inference_steps,
                callback=self._capture_step,
                callback_steps=1  # capture every step
            )

        return self.intermediate_images

    def save_image(self, image: Image.Image, output_path: str):
        """
        Save a single PIL image to the specified path.
        """
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        image.save(output_path)
        print(f"Image saved to {output_path}")
    def save_images(self, images, directory="generated"):
        """
        Save a list of images to the given directory.
        """
        os.makedirs(directory, exist_ok=True)
        for i, img in enumerate(images):
            path = os.path.join(directory, f"step_{i:02d}.png")
            img.save(path)
        print(f"Saved {len(images)} images to '{directory}/'")


In [5]:
if __name__ == "__main__":
    generator = SD15ImageGenerator(num_inference_steps=100)
    prompt = "a fantasy castle floating in the sky, vivid colors, highly detailed"
    
    images = generator.generate_image(prompt)
    
    generator.save_images(images, directory="generated")


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/media/r/Data/AI/AiArtStudio.Puzzle/.venv/lib/python3.12/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:899: FutureWarning: `callback` is deprecated and will be removed in version 1.0.0. Passing `callback` as an input argument to `__call__` is deprecated, consider using `callback_on_step_end`
  deprecate(
/media/r

  0%|          | 0/100 [00:00<?, ?it/s]

Saved 100 images to 'generated/'
